#Import Data

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# Import Model

from keras.models import load_model

loaded_model = load_model('/content/drive/MyDrive/Tubes_IF5171/model_lstm_s2.h5')

In [47]:
# Import Data

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Tubes_IF5171/database_fix.csv")
df.head()

,Date,Close*
0,8/31/2022,89.55
1,8/30/2022,91.64
2,8/29/2022,97.01
3,8/26/2022,93.06
4,8/25/2022,92.52


#Cleansing Data

In [48]:
df['Date'] = pd.to_datetime(df['Date'])

numeric_columns = ['Close*']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

In [49]:
import numpy as np

df = df.fillna(df.mean())


<ipython-input-49-9e44da1e6eb5>:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df = df.fillna(df.mean())


# Membuat data sequence untuk input ke LSTM

In [50]:
df_sorted = df.sort_values(by='Date', ascending=False)
sequence_data = df_sorted.values[:3]
sequence_data

array([[Timestamp('2022-08-31 00:00:00'), 89.55],
       [Timestamp('2022-08-30 00:00:00'), 91.64],
       [Timestamp('2022-08-29 00:00:00'), 97.01]], dtype=object)

### Data sequence dilakuan normalisasi

In [51]:
middle_matrix = sequence_data[:, 1]
print(middle_matrix)

[89.55 91.64 97.01]


In [52]:
middle_matrix = middle_matrix.astype(float)
middle_matrix

array([89.55, 91.64, 97.01])

In [53]:
middle_matrix = middle_matrix.reshape(1, -1)
print(middle_matrix)

[[89.55 91.64 97.01]]


In [54]:
# Normalisasi menggunakan MinMaxScaler

from sklearn.preprocessing import MinMaxScaler
scaler_X = MinMaxScaler(feature_range=(0, 1))

X_normalized = scaler_X.fit_transform(middle_matrix.reshape(-1, 1)).reshape(middle_matrix.shape + (1,))
print(X_normalized)

[[[0.        ]
  [0.28016086]
  [1.        ]]]


In [55]:
X_normalized = X_normalized.astype(np.float32)
X_normalized

array([[[0.        ],
        [0.28016084],
        [1.        ]]], dtype=float32)

#Prediksi

### Ambil last date di database

In [56]:
last_date = df['Date'][:1]
last_date

0   2022-08-31
Name: Date, dtype: datetime64[ns]

In [57]:
last_date = pd.Timestamp(last_date.iloc[0])
last_date

Timestamp('2022-08-31 00:00:00')

## Berapa langkah dari data terakhir yang akan diprediksi (ini input dari user)

In [58]:
# Number of days to forecast
num_days_to_forecast = 5

### Ambil data sequence yang telah dinormalisasi untuk dijadikan input LSTM

In [59]:
input_sequence = X_normalized

### Proses LSTM

In [60]:
# Initialize an array to store the forecasted values
forecasted_values = []

# Loop to forecast for each day
for _ in range(num_days_to_forecast):
    # Make a prediction for the next day
    next_day_prediction = loaded_model.predict(input_sequence).flatten()[0]

    # Store the prediction in the forecasted_values array
    forecasted_values.append(next_day_prediction)

    # Update the input_sequence for the next prediction
    next_day_prediction = np.array([[next_day_prediction]])
    input_sequence = np.concatenate([input_sequence[:, 1:], np.expand_dims(next_day_prediction, axis=1)], axis=1)

# Convert forecasted_values to a numpy array
forecasted_values = np.array(forecasted_values)
print(forecasted_values)


1/1 [==============================] - 0s 18ms/step
[0.21586181 0.393701   0.7024369  0.22116153 0.3599378 ]


### Memberi tanggal ke hasil prediksi (hari Sabtu dan Minggu dilewat)

In [61]:
def next_weekday(d):
    while d.weekday() in {5, 6}:  # 5 adalah Sabtu, 6 adalah Minggu
        d += pd.DateOffset(days=1)
    return d

In [62]:
# Create dates for the forecasted values
forecast_dates = pd.date_range(start=last_date, periods=num_days_to_forecast + 1, freq='B').map(next_weekday)[1:]

# Create a DataFrame to store the forecasted values and dates
forecast_df = pd.DataFrame({
    'Date': forecast_dates,
    'Forecast': forecasted_values
})

# Display the forecast DataFrame
print(forecast_df)

        Date  Forecast
0 2022-09-01  0.215862
1 2022-09-02  0.393701
2 2022-09-05  0.702437
3 2022-09-06  0.221162
4 2022-09-07  0.359938


### Denormalisasi data

In [63]:
# mengembalikan nilai forecast ke harga
reverse_forecast =  scaler_X.inverse_transform(forecasted_values.reshape(-1, 1)).flatten()

reverse_forecast_df = pd.DataFrame({
    'Date': forecast_df['Date'],
    'Reverse_Forecast': reverse_forecast
})

reverse_forecast_df

,Date,Reverse_Forecast
0,2022-09-01,91.160324
1,2022-09-02,92.487007
2,2022-09-05,94.790176
3,2022-09-06,91.199860
4,2022-09-07,92.235138


hasil ini yang ditampilkan ke user